In [ ]:
import numpy as np
import re
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from __future__ import unicode_literals
from imblearn.combine import SMOTETomek
from math import *
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from IPython.core import display as ICD
from xgboost import XGBRegressor,XGBClassifier
from catboost import CatBoostRegressor,CatBoostClassifier
from lightgbm import LGBMClassifier,LGBMRegressor
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score,precision_score
from holidays import CountryHoliday
from imblearn.combine import SMOTEENN
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV,LogisticRegression,ElasticNet
from sklearn.ensemble import BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import eli5
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import classification_report, f1_score,accuracy_score ,confusion_matrix
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import IsolationForest
import matplotlib.gridspec as gridspec
from sklearn.impute import KNNImputer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import seaborn as sns
import warnings
from warnings import simplefilter
import ast
simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
train_df = pd.read_csv("/kaggle/input/playground-series-s3e22/train.csv",na_values=["None","none"])
test_df = pd.read_csv("/kaggle/input/playground-series-s3e22/test.csv",na_values=["None","none"])

In [ ]:
train_df.head()

In [ ]:
train_df.sample(5)

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
train_df.isna().sum()

In [ ]:
for column in train_df.columns[train_df.isna().any()].tolist():
    train_df[column] = train_df[column].fillna(train_df[column].mode()[0])

In [ ]:
train_df.duplicated().sum()

In [ ]:
train_df["surgery"].unique()

In [ ]:
train_df["age"].unique()

In [ ]:
train_df["temp_of_extremities"].unique()

In [ ]:
train_df["peripheral_pulse"].unique()

In [ ]:
train_df["mucous_membrane"].unique()

In [ ]:
train_df["capillary_refill_time"].unique()

In [ ]:
train_df["pain"].unique()

In [ ]:
train_df["peristalsis"].unique()

In [ ]:
train_df["abdominal_distention"].unique()

In [ ]:
train_df["nasogastric_tube"].unique()

In [ ]:
train_df["nasogastric_reflux"].unique()

In [ ]:
train_df["rectal_exam_feces"].unique()

In [ ]:
train_df["abdomen"].unique()

In [ ]:
train_df["abdomo_appearance"].unique()

In [ ]:
train_df["surgical_lesion"].unique()

In [ ]:
train_df["cp_data"].unique()

In [ ]:
train_df["outcome"].unique()

In [ ]:
train_df.describe().T

In [ ]:
train_df.head()

In [ ]:
train_df.drop(["id","hospital_number"],axis=1,inplace=True)

In [ ]:
fig , ax = plt.subplots(4,4,figsize = (40,29))
r = 0
c = 0
for column in train_df.select_dtypes(include="object").drop("outcome",axis=1):
    if r % 2 == 0 :
        if (c == 0) or (c == 2) :
            pal = "viridis"
        else:
            pal = "flare"
    else:
        if (c == 0) or (c == 2) :
            pal = "flare"
        else:
            pal = "viridis"
    ax1 = sns.countplot(x=train_df[column],ax=ax[r,c],palette=pal)
    ax1.tick_params(axis= "x" ,labelsize=20,labelrotation=45)
    ax1.tick_params(axis= "y" ,labelsize=20)
    ax1.set_title(f"({column}) Count",size=20)
    ax1.set_xlabel(None, fontsize=20)
    ax1.set_ylabel(None, fontsize=20)
    c += 1
    if c > 3:
        r+=1
        c=0
    plt.tight_layout()

In [ ]:
fig , ax = plt.subplots(4,4,figsize = (40,29))
r = 0
c = 0
for column in train_df.select_dtypes(include="object").drop("outcome",axis=1):
    ax1 = sns.countplot(x=train_df[column],ax=ax[r,c],hue = train_df["outcome"])
    ax1.tick_params(axis= "x" ,labelsize=20,labelrotation=45)
    ax1.tick_params(axis= "y" ,labelsize=20)
    ax1.set_title(f"({column}) Count",size=20)
    ax1.set_xlabel(None, fontsize=20)
    ax1.set_ylabel(None, fontsize=20)
    ax1.legend(fontsize = 20,loc = "upper right")
    c += 1
    if c > 3:
        r+=1
        c=0
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
target_dict = dict(train_df["outcome"].value_counts())
plt.pie(target_dict.values(),labels=target_dict.keys(),wedgeprops={"edgecolor":"black"},autopct="%1.1f%%")
plt.tight_layout()

In [ ]:
fig , ax = plt.subplots(2,5,figsize = (40,20))
r = 0
c = 0

for column in train_df.select_dtypes(include="number").columns:
    sns.kdeplot(train_df[train_df["outcome"] == "died"][column], alpha=0.5,shade = True,ax=ax[r,c])
    sns.kdeplot(train_df[train_df["outcome"] == "euthanized"][column], alpha=0.5,shade = True,ax=ax[r,c])
    sns.kdeplot(train_df[train_df["outcome"] == "lived"][column], alpha=0.5,shade = True,ax=ax[r,c])
    c+=1
    if c > 4:
        r += 1
        c = 0
fig.legend(title="Outcome",labels = ["Died","Euthanized","Lived"],title_fontsize=20,fontsize=20)
plt.show()
plt.tight_layout()

In [ ]:
my_plot = sns.PairGrid(train_df,hue = "outcome")
my_plot = my_plot.map_diag(sns.kdeplot, shade=True,hue = None)
my_plot = my_plot.map_lower(sns.scatterplot , hue = None)
my_plot.palette = sns.color_palette("Set2", len(train_df["outcome"].unique()))
my_plot = my_plot.map_upper(sns.scatterplot).add_legend(title='Outcome', fontsize= 20 ,title_fontsize = 20, loc = "upper right")
sns.set_context("paper", rc={"axes.labelsize":15, "legend_title.labelsize":2})
plt.tight_layout()

In [ ]:
df_corr = train_df[train_df.select_dtypes(include=np.number).columns.tolist()]
plt.figure(figsize=(12,6))
sns.heatmap(df_corr.corr(),annot=True)
plt.tight_layout()

In [ ]:
train_df.head()

In [ ]:
train_df["surgery"] = train_df["surgery"].replace("no",0).replace("yes",1)
#############
test_df["surgery"] = test_df["surgery"].replace("no",0).replace("yes",1)

In [ ]:
train_df["age"] = train_df["age"].replace("adult",1).replace("young",0)
################
test_df["age"] = test_df["age"].replace("adult",1).replace("young",0)

In [ ]:
dummies = pd.get_dummies(train_df[['mucous_membrane',"peristalsis","rectal_exam_feces","abdomen","abdomo_appearance","temp_of_extremities"]])
train_df = pd.concat([train_df,dummies],axis=1)
train_df.drop(['mucous_membrane',"peristalsis","rectal_exam_feces","abdomen","abdomo_appearance","temp_of_extremities"],axis=1,inplace=True)

In [ ]:
dummies = pd.get_dummies(test_df[['mucous_membrane',"peristalsis","rectal_exam_feces","abdomen","abdomo_appearance","temp_of_extremities"]])
test_df = pd.concat([test_df,dummies],axis=1)
test_df.drop(['mucous_membrane',"peristalsis","rectal_exam_feces","abdomen","abdomo_appearance","temp_of_extremities"],axis=1,inplace=True)

In [ ]:
train_df["peripheral_pulse"] = train_df["peripheral_pulse"].replace("absent",0).replace("reduced",1).replace("normal",2).replace("increased",3)
###############
test_df["peripheral_pulse"] = test_df["peripheral_pulse"].replace("absent",0).replace("reduced",1).replace("normal",2).replace("increased",3)

In [ ]:
train_df["capillary_refill_time"] = train_df["capillary_refill_time"].replace("less_3_sec",0).replace("3",1).replace("more_3_sec",2)
#########
test_df["capillary_refill_time"] = test_df["capillary_refill_time"].replace("less_3_sec",0).replace("3",1).replace("more_3_sec",2)

In [ ]:
train_df["pain"] = train_df["pain"].replace("slight",0).replace("alert",1).replace("mild_pain",2).replace("depressed",3).replace("severe_pain",4).replace("extreme_pain",5)
##########
test_df["pain"] = test_df["pain"].replace("slight",0).replace("alert",1).replace("mild_pain",2).replace("moderate",3).replace("depressed",4).replace("severe_pain",5).replace("extreme_pain",6)

In [ ]:
train_df["abdominal_distention"] = train_df["abdominal_distention"].replace("slight",0).replace("moderate",1).replace("severe",2)
######
test_df["abdominal_distention"] = test_df["abdominal_distention"].replace("slight",0).replace("moderate",1).replace("severe",2)

In [ ]:
train_df["nasogastric_tube"] = train_df["nasogastric_tube"].replace("slight",0).replace("significant",1)
######
test_df["nasogastric_tube"] = test_df["nasogastric_tube"].replace("slight",0).replace("significant",1)

In [ ]:
train_df["nasogastric_reflux"] = train_df["nasogastric_reflux"].replace("slight",0).replace("less_1_liter",1).replace("more_1_liter",2)
########
test_df["nasogastric_reflux"] = test_df["nasogastric_reflux"].replace("slight",0).replace("less_1_liter",1).replace("more_1_liter",2)

In [ ]:
train_df["surgical_lesion"] = train_df["surgical_lesion"].replace("no",0).replace("yes",1)
#######
test_df["surgical_lesion"] = test_df["surgical_lesion"].replace("no",0).replace("yes",1)

In [ ]:
train_df["cp_data"] = train_df["cp_data"].replace("no",0).replace("yes",1)
###########
test_df["cp_data"] = test_df["cp_data"].replace("no",0).replace("yes",1)

In [ ]:
train_df["outcome"] = train_df["outcome"].replace("died",0).replace("euthanized",1).replace("lived",2)

In [ ]:
X = train_df.drop("outcome",axis=1)
y = train_df["outcome"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

## LogisticRegression

In [ ]:
model_log = LogisticRegression()
model_log.fit(X_train,y_train)

In [ ]:
y_pre_log = model_log.predict(X_test)
print(classification_report(y_test,y_pre_log))

In [ ]:
con = confusion_matrix(y_test,y_pre_log)
plt.figure(figsize=(12,6))
sns.heatmap(con,annot=True)
plt.tight_layout()

In [ ]:
C = cross_validate(model_log,X,y,cv=7,return_train_score=True,scoring=('balanced_accuracy','precision_macro','recall_macro','roc_auc_ovo_weighted','roc_auc_ovr_weighted',"f1_macro"))
time = C["score_time"].mean()
print(f"The Time is : {time}")
tr_score = C["train_balanced_accuracy"].mean()
print(f"Train Score is : {tr_score}")
ts_score = C["test_balanced_accuracy"].mean()
print(f"Test Score is : {ts_score}")
precision_score = C["test_precision_macro"].mean()
print(f"Precision Score is : {precision_score}")
recall_score = C["test_recall_macro"].mean()
print(f"Recall Score is : {recall_score}")
f1_score = C["test_f1_macro"].mean()
print(f"F1 Score is : {f1_score}")
OVO_score = C["test_roc_auc_ovo_weighted"].mean()
print(f"One vs One Score is : {OVO_score}")
OVR_score = C["test_roc_auc_ovr_weighted"].mean()
print(f"One vs Rest Score is : {OVR_score}")

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
Log=OneVsRestClassifier(LogisticRegression())
Log.fit(X_train,y_train)
y_pred =Log.predict(X_test)
pred_prob = Log.predict_proba(X_test)
classes=np.unique(y_test)
y_test_binarized=label_binarize(y_test,classes=np.unique(y_test))
fpr = {}
tpr = {}
thresh ={}
roc_auc = dict()

n_class = classes.shape[0]

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test_binarized[:,i], pred_prob[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], linestyle='--', 
             label='Class %s vs Rest (AUC=%0.2f)'%(classes[i],roc_auc[i]))
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()

## RandomForestClassifier

In [ ]:
model_rand = RandomForestClassifier()
model_rand.fit(X_train,y_train)

In [ ]:
y_pre_rand= model_rand.predict(X_test)
print(classification_report(y_test,y_pre_rand))

In [ ]:
con = confusion_matrix(y_test,y_pre_rand)
plt.figure(figsize=(12,6))
sns.heatmap(con,annot=True)
plt.tight_layout()

In [ ]:
C = cross_validate(model_rand,X,y,cv=7,return_train_score=True,scoring=('balanced_accuracy','precision_macro','recall_macro','roc_auc_ovo_weighted','roc_auc_ovr_weighted',"f1_macro"))
time = C["score_time"].mean()
print(f"The Time is : {time}")
tr_score = C["train_balanced_accuracy"].mean()
print(f"Train Score is : {tr_score}")
ts_score = C["test_balanced_accuracy"].mean()
print(f"Test Score is : {ts_score}")
precision_score = C["test_precision_macro"].mean()
print(f"Precision Score is : {precision_score}")
recall_score = C["test_recall_macro"].mean()
print(f"Recall Score is : {recall_score}")
f1_score = C["test_f1_macro"].mean()
print(f"F1 Score is : {f1_score}")
OVO_score = C["test_roc_auc_ovo_weighted"].mean()
print(f"One vs One Score is : {OVO_score}")
OVR_score = C["test_roc_auc_ovr_weighted"].mean()
print(f"One vs Rest Score is : {OVR_score}")

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
rand=OneVsRestClassifier(RandomForestClassifier())
rand.fit(X_train,y_train)
y_pred =rand.predict(X_test)
pred_prob = rand.predict_proba(X_test)
classes=np.unique(y_test)
y_test_binarized=label_binarize(y_test,classes=np.unique(y_test))
fpr = {}
tpr = {}
thresh ={}
roc_auc = dict()

n_class = classes.shape[0]

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test_binarized[:,i], pred_prob[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], linestyle='--', 
             label='Class %s vs Rest (AUC=%0.2f)'%(classes[i],roc_auc[i]))
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()

## GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
model_na = GaussianNB()
model_na.fit(X_train,y_train)

In [ ]:
y_pre_na= model_na.predict(X_test)
print(classification_report(y_test,y_pre_na))

In [ ]:
con = confusion_matrix(y_test,y_pre_na)
plt.figure(figsize=(12,6))
sns.heatmap(con,annot=True)
plt.tight_layout()

In [ ]:
C = cross_validate(model_na,X,y,cv=7,return_train_score=True,scoring=('balanced_accuracy','precision_macro','recall_macro','roc_auc_ovo_weighted','roc_auc_ovr_weighted',"f1_macro"))
time = C["score_time"].mean()
print(f"The Time is : {time}")
tr_score = C["train_balanced_accuracy"].mean()
print(f"Train Score is : {tr_score}")
ts_score = C["test_balanced_accuracy"].mean()
print(f"Test Score is : {ts_score}")
precision_score = C["test_precision_macro"].mean()
print(f"Precision Score is : {precision_score}")
recall_score = C["test_recall_macro"].mean()
print(f"Recall Score is : {recall_score}")
f1_score = C["test_f1_macro"].mean()
print(f"F1 Score is : {f1_score}")
OVO_score = C["test_roc_auc_ovo_weighted"].mean()
print(f"One vs One Score is : {OVO_score}")
OVR_score = C["test_roc_auc_ovr_weighted"].mean()
print(f"One vs Rest Score is : {OVR_score}")

In [ ]:
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
Na=OneVsRestClassifier(GaussianNB())
Na.fit(X_train,y_train)
y_pred =Na.predict(X_test)
pred_prob = Na.predict_proba(X_test)
classes=np.unique(y_test)
y_test_binarized=label_binarize(y_test,classes=np.unique(y_test))
fpr = {}
tpr = {}
thresh ={}
roc_auc = dict()

n_class = classes.shape[0]

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test_binarized[:,i], pred_prob[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], linestyle='--', 
             label='Class %s vs Rest (AUC=%0.2f)'%(classes[i],roc_auc[i]))
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()

## KNeighborsClassifier

In [ ]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train,y_train)

In [ ]:
y_pre_knn= model_knn.predict(X_test)
print(classification_report(y_test,y_pre_knn))

In [ ]:
con = confusion_matrix(y_test,y_pre_knn)
plt.figure(figsize=(12,6))
sns.heatmap(con,annot=True)
plt.tight_layout()

In [ ]:
C = cross_validate(model_knn,X,y,cv=7,return_train_score=True,scoring=('balanced_accuracy','precision_macro','recall_macro','roc_auc_ovo_weighted','roc_auc_ovr_weighted',"f1_macro"))
time = C["score_time"].mean()
print(f"The Time is : {time}")
tr_score = C["train_balanced_accuracy"].mean()
print(f"Train Score is : {tr_score}")
ts_score = C["test_balanced_accuracy"].mean()
print(f"Test Score is : {ts_score}")
precision_score = C["test_precision_macro"].mean()
print(f"Precision Score is : {precision_score}")
recall_score = C["test_recall_macro"].mean()
print(f"Recall Score is : {recall_score}")
f1_score = C["test_f1_macro"].mean()
print(f"F1 Score is : {f1_score}")
OVO_score = C["test_roc_auc_ovo_weighted"].mean()
print(f"One vs One Score is : {OVO_score}")
OVR_score = C["test_roc_auc_ovr_weighted"].mean()
print(f"One vs Rest Score is : {OVR_score}")

In [ ]:
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
knn=OneVsRestClassifier(KNeighborsClassifier())
knn.fit(X_train,y_train)
y_pred =knn.predict(X_test)
pred_prob = knn.predict_proba(X_test)
classes=np.unique(y_test)
y_test_binarized=label_binarize(y_test,classes=np.unique(y_test))
fpr = {}
tpr = {}
thresh ={}
roc_auc = dict()

n_class = classes.shape[0]

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test_binarized[:,i], pred_prob[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], linestyle='--', 
             label='Class %s vs Rest (AUC=%0.2f)'%(classes[i],roc_auc[i]))
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()

## GradientBoostingClassifier

In [ ]:
model_grad = GradientBoostingClassifier()
model_grad.fit(X_train,y_train)

In [ ]:
y_pre_grad= model_grad.predict(X_test)
print(classification_report(y_test,y_pre_grad))

In [ ]:
con = confusion_matrix(y_test,y_pre_grad)
plt.figure(figsize=(12,6))
sns.heatmap(con,annot=True)
plt.tight_layout()

In [ ]:
C = cross_validate(model_grad,X,y,cv=7,return_train_score=True,scoring=('balanced_accuracy','precision_macro','recall_macro','roc_auc_ovo_weighted','roc_auc_ovr_weighted',"f1_macro"))
time = C["score_time"].mean()
print(f"The Time is : {time}")
tr_score = C["train_balanced_accuracy"].mean()
print(f"Train Score is : {tr_score}")
ts_score = C["test_balanced_accuracy"].mean()
print(f"Test Score is : {ts_score}")
precision_score = C["test_precision_macro"].mean()
print(f"Precision Score is : {precision_score}")
recall_score = C["test_recall_macro"].mean()
print(f"Recall Score is : {recall_score}")
f1_score = C["test_f1_macro"].mean()
print(f"F1 Score is : {f1_score}")
OVO_score = C["test_roc_auc_ovo_weighted"].mean()
print(f"One vs One Score is : {OVO_score}")
OVR_score = C["test_roc_auc_ovr_weighted"].mean()
print(f"One vs Rest Score is : {OVR_score}")

In [ ]:
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
grad=OneVsRestClassifier(GradientBoostingClassifier())
grad.fit(X_train,y_train)
y_pred =grad.predict(X_test)
pred_prob = grad.predict_proba(X_test)
classes=np.unique(y_test)
y_test_binarized=label_binarize(y_test,classes=np.unique(y_test))
fpr = {}
tpr = {}
thresh ={}
roc_auc = dict()

n_class = classes.shape[0]

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test_binarized[:,i], pred_prob[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], linestyle='--', 
             label='Class %s vs Rest (AUC=%0.2f)'%(classes[i],roc_auc[i]))
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()

## LGBMClassifier

In [ ]:
model_lg = LGBMClassifier()
model_lg.fit(X_train,y_train)

In [ ]:
y_pre_lg= model_lg.predict(X_test)
print(classification_report(y_test,y_pre_lg))

In [ ]:
con = confusion_matrix(y_test,y_pre_lg)
plt.figure(figsize=(12,6))
sns.heatmap(con,annot=True)
plt.tight_layout()

In [ ]:
C = cross_validate(model_lg,X,y,cv=7,return_train_score=True,scoring=('balanced_accuracy','precision_macro','recall_macro','roc_auc_ovo_weighted','roc_auc_ovr_weighted',"f1_macro"))
time = C["score_time"].mean()
print(f"The Time is : {time}")
tr_score = C["train_balanced_accuracy"].mean()
print(f"Train Score is : {tr_score}")
ts_score = C["test_balanced_accuracy"].mean()
print(f"Test Score is : {ts_score}")
precision_score = C["test_precision_macro"].mean()
print(f"Precision Score is : {precision_score}")
recall_score = C["test_recall_macro"].mean()
print(f"Recall Score is : {recall_score}")
f1_score = C["test_f1_macro"].mean()
print(f"F1 Score is : {f1_score}")
OVO_score = C["test_roc_auc_ovo_weighted"].mean()
print(f"One vs One Score is : {OVO_score}")
OVR_score = C["test_roc_auc_ovr_weighted"].mean()
print(f"One vs Rest Score is : {OVR_score}")

In [ ]:
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
lg=OneVsRestClassifier(LGBMClassifier())
lg.fit(X_train,y_train)
y_pred =lg.predict(X_test)
pred_prob = lg.predict_proba(X_test)
classes=np.unique(y_test)
y_test_binarized=label_binarize(y_test,classes=np.unique(y_test))
fpr = {}
tpr = {}
thresh ={}
roc_auc = dict()

n_class = classes.shape[0]

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test_binarized[:,i], pred_prob[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], linestyle='--', 
             label='Class %s vs Rest (AUC=%0.2f)'%(classes[i],roc_auc[i]))
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()

## XGBClassifier

In [ ]:
y_train_xg = y_train.replace("died",0).replace("euthanized",1).replace("lived",2)
y_test_xg = y_test.replace("died",0).replace("euthanized",1).replace("lived",2)
model_xg = XGBClassifier()
model_xg.fit(X_train,y_train_xg)

In [ ]:
y_pre_xg= model_xg.predict(X_test)
print(classification_report(y_test_xg,y_pre_xg))

In [ ]:
con = confusion_matrix(y_test_xg,y_pre_xg)
plt.figure(figsize=(12,6))
sns.heatmap(con,annot=True)
plt.tight_layout()

In [ ]:
y_xg = y.replace("died",0).replace("euthanized",1).replace("lived",2)
C = cross_validate(model_xg,X,y_xg,cv=7,return_train_score=True,scoring=('balanced_accuracy','precision_macro','recall_macro','roc_auc_ovo_weighted','roc_auc_ovr_weighted',"f1_macro"))
time = C["score_time"].mean()
print(f"The Time is : {time}")
tr_score = C["train_balanced_accuracy"].mean()
print(f"Train Score is : {tr_score}")
ts_score = C["test_balanced_accuracy"].mean()
print(f"Test Score is : {ts_score}")
precision_score = C["test_precision_macro"].mean()
print(f"Precision Score is : {precision_score}")
recall_score = C["test_recall_macro"].mean()
print(f"Recall Score is : {recall_score}")
f1_score = C["test_f1_macro"].mean()
print(f"F1 Score is : {f1_score}")
OVO_score = C["test_roc_auc_ovo_weighted"].mean()
print(f"One vs One Score is : {OVO_score}")
OVR_score = C["test_roc_auc_ovr_weighted"].mean()
print(f"One vs Rest Score is : {OVR_score}")

In [ ]:
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
xg=OneVsRestClassifier(XGBClassifier())
xg.fit(X_train,y_train_xg)
y_pred =xg.predict(X_test)
pred_prob = xg.predict_proba(X_test)
classes=np.unique(y_test)
y_test_binarized=label_binarize(y_test,classes=np.unique(y_test))
fpr = {}
tpr = {}
thresh ={}
roc_auc = dict()

n_class = classes.shape[0]

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test_binarized[:,i], pred_prob[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], linestyle='--', 
             label='Class %s vs Rest (AUC=%0.2f)'%(classes[i],roc_auc[i]))
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()

## CatBoostClassifier

In [ ]:
model_cat = CatBoostClassifier()
model_cat.fit(X_train,y_train)

In [ ]:
y_pre_cat= model_cat.predict(X_test)
print(classification_report(y_test,y_pre_cat))

In [ ]:
con = confusion_matrix(y_test,y_pre_cat)
plt.figure(figsize=(12,6))
sns.heatmap(con,annot=True)
plt.tight_layout()

In [ ]:
C = cross_validate(model_cat,X,y,cv=7,return_train_score=True,scoring=('balanced_accuracy','precision_macro','recall_macro','roc_auc_ovo_weighted','roc_auc_ovr_weighted',"f1_macro"))

In [ ]:
time = C["score_time"].mean()
print(f"The Time is : {time}")
tr_score = C["train_balanced_accuracy"].mean()
print(f"Train Score is : {tr_score}")
ts_score = C["test_balanced_accuracy"].mean()
print(f"Test Score is : {ts_score}")
precision_score = C["test_precision_macro"].mean()
print(f"Precision Score is : {precision_score}")
recall_score = C["test_recall_macro"].mean()
print(f"Recall Score is : {recall_score}")
f1_score = C["test_f1_macro"].mean()
print(f"F1 Score is : {f1_score}")
OVO_score = C["test_roc_auc_ovo_weighted"].mean()
print(f"One vs One Score is : {OVO_score}")
OVR_score = C["test_roc_auc_ovr_weighted"].mean()
print(f"One vs Rest Score is : {OVR_score}")

In [ ]:
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.metrics import auc
from sklearn.preprocessing import label_binarize
cat=OneVsRestClassifier(CatBoostClassifier())
cat.fit(X_train,y_train)

In [ ]:
y_pred =cat.predict(X_test)
pred_prob = cat.predict_proba(X_test)
classes=np.unique(y_test)
y_test_binarized=label_binarize(y_test,classes=np.unique(y_test))
fpr = {}
tpr = {}
thresh ={}
roc_auc = dict()

n_class = classes.shape[0]

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test_binarized[:,i], pred_prob[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], linestyle='--', 
             label='Class %s vs Rest (AUC=%0.2f)'%(classes[i],roc_auc[i]))
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()

In [ ]:
model = XGBClassifier(
    n_estimators = 1250,
    max_depth= 4,
    learning_rate= 0.4, 
    colsample_bytree= 0.68, 
    objective="multi:softmax",
    random_state=1,
)

In [ ]:
model.fit(X,y)

In [ ]:
test_df.head()

In [ ]:
ids = test_df["id"]

In [ ]:
test_df.isna().sum()

In [ ]:
for column in test_df.columns[test_df.isna().any()].tolist():
    test_df[column] = test_df[column].fillna(test_df[column].mode()[0])

In [ ]:
test_df.drop(["id","hospital_number"],axis=1,inplace=True)

In [ ]:
test_df["peristalsis_distend_small"] = 0
test_df["rectal_exam_feces_serosanguious"] = 0

In [ ]:
test_df = test_df[X.columns]

In [ ]:
test_df.head()

In [ ]:
Prediction = model.predict(test_df)

In [ ]:
Predictions = pd.DataFrame({"id":ids,"outcome":Prediction.reshape(-1)})
Predictions["outcome"] = Predictions["outcome"].replace(0,"died").replace(1,"euthanized").replace(2,"lived")

In [ ]:
Predictions

In [ ]:
Predictions.to_csv("submission.csv",index=False)